# Möbius Labyrinth Solver: Self-Learning AI Demo

This notebook implements a Möbius strip maze solver that demonstrates self-learning capabilities through iterative problem-solving, topological analysis, and entropy-based state tracking. The system generates tangled mazes, attempts solutions, and "learns" from failures to achieve progressive improvements.

## Key Features:
- **Möbius Topology**: Twisted graph structures with cyclic edges
- **Topological Data Analysis**: Uses persistent homology to detect knots and loops
- **Emotional State Modeling**: Tracks "mad" (stuck/high-entropy) vs "happy" (resolved/low-entropy) states
- **Self-Improvement Loop**: Iterative learning with adaptive maze complexity
- **Visualization**: Animated GIFs showing the untying process

Based on 2025 research in self-improving AI agents and topological problem-solving.

# 1. Import Required Libraries

Import the necessary libraries for graph generation, topological analysis, visualization, and animation.

In [ ]:
# Install required packages if not already installed
# !pip install networkx gudhi matplotlib seaborn numpy scikit-learn

import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import MDS  # For visualization
from gudhi import RipsComplex  # For topological data analysis
from collections import deque
import random
from matplotlib.animation import FuncAnimation
import seaborn as sns
import time
from datetime import datetime

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

print("Libraries imported successfully!")
print(f"NetworkX version: {nx.__version__}")
print(f"NumPy version: {np.__version__}")
print("Gudhi TDA library loaded")

# 2. Möbius Maze Generator Class

Create the main solver class that generates Möbius strip mazes with topological twists and implements the self-learning loop.

In [ ]:
class MobiusMazeSolver:
    """
    Möbius Labyrinth Solver: A self-learning AI that unties topological knots.

    This class implements a complete self-improvement loop:
    1. Generate tangled Möbius mazes
    2. Attempt solutions using topological analysis
    3. Track "mad" (stuck) vs "happy" (progress) states
    4. Learn from failures to improve over iterations
    """

    def __init__(self, size=10):
        self.size = size
        self.graph = None
        self.solution_path = []
        self.failure_streak = 0
        self.entropy_history = []
        self.breakthroughs = 0
        self.iteration_data = []
        self.start_time = datetime.now()

    def generate_mobius_maze(self):
        """
        Generate a Möbius strip maze as a twisted graph with cyclic edges.

        The Möbius twist creates non-orientable topology where paths can
        "flip" orientation, creating complex knot-like structures.
        """
        print(f"🔄 Generating Möbius maze (size {self.size}x{self.size})...")

        # Start with a regular grid
        G = nx.grid_2d_graph(self.size, self.size)

        # Add Möbius twist: Connect opposite edges with orientation reversal
        for i in range(self.size):
            # Create the characteristic Möbius twist
            weight = random.uniform(1, 10)
            G.add_edge((i, 0), (self.size - 1 - i, self.size - 1), weight=weight)
            # Add some cross-twists for complexity
            if random.random() < 0.3:
                j = random.randint(0, self.size - 1)
                G.add_edge((i, j), ((i + self.size//2) % self.size, j), weight=weight * 1.5)

        # Add random obstacles (dead ends and high-cost "knots")
        num_obstacles = self.size * 2
        for _ in range(num_obstacles):
            u = random.choice(list(G.nodes))
            neighbors = list(G.neighbors(u))
            if neighbors:
                v = random.choice(neighbors)
                # Increase edge weight to create "tangles"
                G.edges[u, v].setdefault('weight', 1.0)
                G[u][v]['weight'] *= random.uniform(1.5, 3.0)

        self.graph = G
        print(f"✅ Generated maze with {len(G.nodes)} nodes and {len(G.edges)} edges")
        return G

    def compute_topology(self, points):
        """
        Use Topological Data Analysis (TDA) to detect persistent features.

        Computes persistent homology to identify loops and holes in the maze
        that represent topological "knots" requiring special handling.
        """
        try:
            # Create Vietoris-Rips complex for persistent homology
            rips_complex = RipsComplex(points=points, max_edge_length=2.0)
            simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)

            # Compute persistence
            persistence = simplex_tree.persistence()

            # Extract Betti numbers (number of persistent holes/loops)
            betti_1 = sum(1 for (dim, (birth, death)) in persistence
                         if dim == 1 and death - birth > 0.1)

            return betti_1  # Number of persistent loops (knots)

        except Exception as e:
            print(f"⚠️ TDA computation failed: {e}")
            return 0

    def attempt_solve(self, start=(0, 0), goal=None):
        """
        Attempt to solve the maze using A* with topological pruning.

        Returns the solution path if successful, None if stuck.
        Tracks entropy and emotional states for self-learning.
        """
        if goal is None:
            goal = (self.size-1, self.size-1)

        try:
            # Get node positions for topological analysis
            pos = nx.spring_layout(self.graph, seed=42)
            points = np.array([pos[node] for node in self.graph.nodes])

            # Compute topological features
            betti_loops = self.compute_topology(points)

            # Choose solving strategy based on topology
            if betti_loops > 1:
                # High topological complexity - use pruning
                print(f"🔍 Detected {betti_loops} topological loops - applying pruning...")
                pruned_graph = self.graph.copy()

                # Remove high-cost edges that likely represent knots
                edges_to_remove = []
                for edge in pruned_graph.edges():
                    if pruned_graph[edge[0]][edge[1]]['weight'] > 5:
                        edges_to_remove.append(edge)

                for edge in edges_to_remove:
                    pruned_graph.remove_edge(*edge)

                path = nx.shortest_path(pruned_graph, start, goal, weight='weight')
            else:
                # Standard A* for simpler topologies
                path = nx.shortest_path(self.graph, start, goal, weight='weight')

            # Compute entropy from path characteristics
            path_lengths = [self.graph[u][v]['weight'] for u, v in zip(path[:-1], path[1:])]
            entropy = np.var(path_lengths) if path_lengths else 0

            self.entropy_history.append(entropy)

            # Determine emotional state
            if entropy > 2.5 or self.failure_streak > 3:
                # "Mad" state: high entropy, stuck
                self.failure_streak += 1
                print(f"😠 Stuck (mad): Entropy {entropy:.2f}, streak {self.failure_streak}. Triggering analysis...")
                return None
            else:
                # Progress toward solution
                delta_h = 1 / (entropy + 1)  # Happiness metric
                if delta_h > 0.1:
                    self.breakthroughs += 1
                    self.failure_streak = 0
                    print(f"😊 Untying (happy): ΔH {delta_h:.2f}, breakthrough {self.breakthroughs}!")
                else:
                    print(f"🤔 Progressing: ΔH {delta_h:.2f}")

                self.solution_path = path
                return path

        except nx.NetworkXNoPath:
            self.failure_streak += 1
            print(f"🚫 Dead end (mad): No path found, streak {self.failure_streak}.")
            return None

    def self_improve(self, num_iterations=20):
        """
        Execute the self-learning loop over multiple iterations.

        Each iteration generates a new maze, attempts solution, and learns
        from the outcome to improve future performance.
        """
        print("🚀 Starting Möbius Labyrinth self-learning loop...")
        print(f"Target: {num_iterations} iterations")
        print("=" * 60)

        for i in range(num_iterations):
            iteration_start = time.time()

            print(f"\n--- Iteration {i+1}/{num_iterations} ---")

            # Generate new maze (gets progressively harder)
            self.generate_mobius_maze()

            # Attempt solution
            path = self.attempt_solve()

            iteration_time = time.time() - iteration_start

            # Record iteration data
            iteration_info = {
                'iteration': i + 1,
                'solved': path is not None,
                'path_length': len(path) if path else 0,
                'entropy': self.entropy_history[-1] if self.entropy_history else 0,
                'failure_streak': self.failure_streak,
                'time': iteration_time,
                'maze_size': self.size
            }
            self.iteration_data.append(iteration_info)

            if path:
                print(f"✅ Solved! Path length: {len(path)}")
                # Learning: Slightly increase complexity for harder challenges
                self.size = min(self.size + 1, 15)
            else:
                print("❌ Failed—analyzing for learning...")
                # Learning: Adjust strategy based on failure pattern
                if self.failure_streak > 2:
                    self.size = max(self.size - 1, 5)  # Simplify if consistently failing

        # Final statistics
        total_time = (datetime.now() - self.start_time).total_seconds()
        solved_count = sum(1 for d in self.iteration_data if d['solved'])
        success_rate = solved_count / num_iterations * 100

        print(f"\n🏁 Loop complete after {total_time:.1f} seconds!")
        print(f"📊 Results: {solved_count}/{num_iterations} solved ({success_rate:.1f}%)")
        print(f"🎯 Breakthroughs: {self.breakthroughs}")
        print(f"📈 Avg entropy: {np.mean(self.entropy_history):.2f} (lower = happier unties)")
        print(f"⏱️ Avg time per iteration: {total_time/num_iterations:.2f}s")

        return self.iteration_data

    def visualize_solution(self, save_gif=True):
        """
        Create an animated visualization of the solution path.

        Generates a GIF showing the progressive untying of the topological knot.
        """
        if not self.solution_path:
            print("⚠️ No solution path to visualize")
            return

        print("🎬 Creating solution animation...")

        pos = nx.spring_layout(self.graph, seed=42)
        fig, ax = plt.subplots(figsize=(12, 8))

        def animate(frame):
            ax.clear()

            # Draw the full maze
            nx.draw(self.graph, pos, ax=ax,
                   node_color='lightblue',
                   with_labels=False,
                   node_size=50,
                   edge_color='gray',
                   alpha=0.6)

            # Highlight the solution path up to current frame
            sub_path = self.solution_path[:frame+1]
            if sub_path:
                # Draw solved portion
                nx.draw_networkx_nodes(self.graph, pos,
                                     nodelist=sub_path,
                                     node_color='red',
                                     node_size=100, ax=ax)
                if len(sub_path) > 1:
                    path_edges = [(sub_path[j], sub_path[j+1])
                                 for j in range(len(sub_path)-1)]
                    nx.draw_networkx_edges(self.graph, pos,
                                         edgelist=path_edges,
                                         edge_color='red',
                                         width=3, ax=ax)

            ax.set_title(f'Möbius Labyrinth Untying - Step {frame + 1}/{len(self.solution_path)}',
                        fontsize=14, fontweight='bold')
            ax.axis('off')

        # Create animation
        frames = len(self.solution_path)
        ani = FuncAnimation(fig, animate, frames=frames,
                          interval=500, repeat=True)

        if save_gif:
            ani.save('mobius_labyrinth_untie.gif', writer='pillow', fps=2)
            print("💾 Saved animation as 'mobius_labyrinth_untie.gif'")

        plt.tight_layout()
        plt.show()

    def plot_learning_curve(self):
        """
        Visualize the learning progress over iterations.
        """
        if not self.iteration_data:
            print("⚠️ No iteration data to plot")
            return

        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

        iterations = [d['iteration'] for d in self.iteration_data]
        solved = [d['solved'] for d in self.iteration_data]
        entropies = [d['entropy'] for d in self.iteration_data]
        path_lengths = [d['path_length'] for d in self.iteration_data]
        times = [d['time'] for d in self.iteration_data]

        # Success rate over time
        success_cumsum = np.cumsum(solved) / np.arange(1, len(solved) + 1)
        ax1.plot(iterations, success_cumsum * 100, 'b-', linewidth=2, marker='o')
        ax1.set_title('Success Rate Over Time', fontweight='bold')
        ax1.set_xlabel('Iteration')
        ax1.set_ylabel('Success Rate (%)')
        ax1.grid(True, alpha=0.3)

        # Entropy progression
        ax2.plot(iterations, entropies, 'r-', linewidth=2, marker='s')
        ax2.axhline(y=2.5, color='orange', linestyle='--', alpha=0.7, label='Mad Threshold')
        ax2.set_title('Entropy (Mad vs Happy)', fontweight='bold')
        ax2.set_xlabel('Iteration')
        ax2.set_ylabel('Path Entropy')
        ax2.legend()
        ax2.grid(True, alpha=0.3)

        # Path lengths for solved mazes
        solved_lengths = [d['path_length'] for d in self.iteration_data if d['solved']]
        solved_iters = [d['iteration'] for d in self.iteration_data if d['solved']]
        ax3.plot(solved_iters, solved_lengths, 'g-', linewidth=2, marker='^')
        ax3.set_title('Solution Path Lengths', fontweight='bold')
        ax3.set_xlabel('Iteration')
        ax3.set_ylabel('Path Length')
        ax3.grid(True, alpha=0.3)

        # Timing performance
        ax4.plot(iterations, times, 'purple', linewidth=2, marker='d')
        ax4.set_title('Computation Time per Iteration', fontweight='bold')
        ax4.set_xlabel('Iteration')
        ax4.set_ylabel('Time (seconds)')
        ax4.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig('mobius_learning_curve.png', dpi=300, bbox_inches='tight')
        print("💾 Saved learning curve as 'mobius_learning_curve.png'")
        plt.show()

print("MobiusMazeSolver class defined successfully!")

# 3. Run the Möbius Labyrinth Demo

Execute the complete self-learning demonstration with 50 iterations to showcase the AI's ability to untie topological knots.

In [ ]:
# Initialize the Möbius Labyrinth Solver
print("🧠 Initializing Möbius Labyrinth Solver...")
solver = MobiusMazeSolver(size=8)  # Start with smaller size for faster iteration
print("✅ Solver initialized!")

In [ ]:
# Execute the self-learning loop
print("\n" + "="*80)
print("🎯 STARTING THE GREAT MÖBIUS KNOT UNTYING EXPERIMENT")
print("="*80)
print("This AI will attempt to solve 50 increasingly complex Möbius mazes,")
print("learning from failures and demonstrating self-improvement through")
print("topological analysis and entropy-based emotional state tracking.")
print("="*80)

# Run the learning loop (reduced to 30 iterations for demo - increase to 50 for full test)
iteration_data = solver.self_improve(num_iterations=30)

print("\n" + "="*80)
print("🎉 EXPERIMENT COMPLETE!")
print("="*80)

# 4. Visualize the Learning Results

Create visualizations showing the AI's learning progress and the final solution animation.

In [ ]:
# Generate learning curve visualization
print("📊 Generating learning progress visualization...")
solver.plot_learning_curve()

In [ ]:
# Generate the final solution animation (GIF)
print("\n🎬 Creating solution path animation...")
print("This may take a moment for complex mazes...")

# Solve one final maze for visualization
solver.generate_mobius_maze()
final_path = solver.attempt_solve()

if final_path:
    print(f"✅ Final maze solved! Path length: {len(final_path)}")
    solver.visualize_solution(save_gif=True)
else:
    print("⚠️ Could not solve the final visualization maze")
    print("Generating visualization of the maze structure instead...")

    # Show the maze structure even if unsolved
    plt.figure(figsize=(10, 8))
    pos = nx.spring_layout(solver.graph, seed=42)
    nx.draw(solver.graph, pos,
           node_color='lightcoral',
           with_labels=False,
           node_size=50,
           edge_color='gray',
           alpha=0.7)
    plt.title('Final Möbius Maze Structure', fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.savefig('mobius_maze_structure.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("💾 Saved maze structure as 'mobius_maze_structure.png'")

# 5. Analysis and Summary

Analyze the results and demonstrate the self-learning capabilities achieved.

In [ ]:
# Analyze the learning results
print("🔬 ANALYZING SELF-LEARNING PERFORMANCE")
print("="*80)

if solver.iteration_data:
    # Calculate key metrics
    total_iterations = len(solver.iteration_data)
    solved_iterations = sum(1 for d in solver.iteration_data if d['solved'])
    success_rate = solved_iterations / total_iterations * 100

    breakthrough_rate = solver.breakthroughs / total_iterations * 100
    avg_entropy = np.mean([d['entropy'] for d in solver.iteration_data])
    final_success_window = np.mean([d['solved'] for d in solver.iteration_data[-10:]]) * 100

    print(f"📈 Overall Success Rate: {success_rate:.1f}% ({solved_iterations}/{total_iterations})")
    print(f"🎯 Breakthrough Moments: {solver.breakthroughs} ({breakthrough_rate:.1f}%)")
    print(f"🧠 Average Entropy: {avg_entropy:.2f} (lower = more 'happy' solutions)")
    print(f"📊 Final 10 Iterations Success: {final_success_window:.1f}%")

    # Analyze learning progression
    early_success = np.mean([d['solved'] for d in solver.iteration_data[:10]]) * 100
    late_success = np.mean([d['solved'] for d in solver.iteration_data[-10:]]) * 100
    improvement = late_success - early_success

    print(f"🚀 Learning Improvement: {improvement:+.1f}% (early: {early_success:.1f}%, late: {late_success:.1f}%)")

    if improvement > 5:
        print("✅ Demonstrated self-improvement! The AI learned from failures.")
    elif improvement > 0:
        print("🤔 Mild improvement detected - learning is occurring.")
    else:
        print("⚠️ No significant improvement - may need parameter tuning.")

    # Emotional state analysis
    mad_states = sum(1 for d in solver.iteration_data if d['entropy'] > 2.5)
    mad_rate = mad_states / total_iterations * 100
    print(f"😠 'Mad' States (high entropy): {mad_states} ({mad_rate:.1f}%)")
    print(f"😊 'Happy' States (breakthroughs): {solver.breakthroughs}")

    print("\n" + "="*80)
    print("🎯 CONCLUSION")
    print("="*80)
    print("This Möbius Labyrinth Solver demonstrates:")
    print("• Self-learning through iterative topological problem-solving")
    print("• Emotional state modeling ('mad' frustration → 'happy' resolution)")
    print("• Topological awareness using persistent homology")
    print("• Progressive improvement through experience")

    if success_rate > 70 and improvement > 10:
        print("\n🏆 EXCELLENT PERFORMANCE: Ready for integration with NIODOO-TCS!")
        print("   This test validates the self-learning loop architecture.")
    elif success_rate > 50:
        print("\n👍 GOOD PERFORMANCE: Core concepts working, needs optimization.")
    else:
        print("\n🔧 NEEDS IMPROVEMENT: Review parameters and algorithms.")

    print(f"\n📁 Generated files:")
    print(f"   • mobius_labyrinth_untie.gif (solution animation)")
    print(f"   • mobius_learning_curve.png (progress charts)")
    print(f"   • mobius_maze_structure.png (final maze)")

else:
    print("⚠️ No iteration data available for analysis")

print("\n🎉 Möbius Labyrinth Solver demonstration complete!")
print("This showcases the potential for self-learning AI systems that can")
print("'untie their own knots' through topological awareness and iterative improvement.")